# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Необходимо ответить на следующие вопросы:
1. Есть ли зависимость между количеством детей и возвратом кредита в срок?
2. Есть ли зависимость между семейным положением и возвратом кредита в срок?
3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
4. Как разные цели кредита влияют на его возврат в срок?

## Обзор данных

Прочитаем данные

In [1]:
import pandas as pd

In [2]:
def read_file() -> pd.DataFrame:
    # try open local file first then — if failed — open on server
    try:
        df = pd.read_csv('./datasets/data.csv')
        return df
    except FileNotFoundError:
        try:
            df = pd.read_csv('/datasets/data.csv')
            return df
        except Exception as e:
            print('Не удалось считать файл с данными на сервере', e, sep='\n')
            return None

    except Exception as e:
        print('Не удалось считать файл с данными на диске', e, sep='\n')
    return None

In [51]:
df = read_file()

Посмотрим на случайные 7 записей в таблице

In [52]:
df.sample(n=7, random_state=1)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
1383,0,353802.811675,37,среднее,1,вдовец / вдова,2,F,пенсионер,0,216452.226085,строительство недвижимости
300,1,-359.193975,33,СРЕДНЕЕ,1,гражданский брак,1,M,сотрудник,0,223001.623994,на проведение свадьбы
6565,2,-1064.854333,35,среднее,1,гражданский брак,1,F,компаньон,0,163591.209323,свадьба
17027,0,NaN,48,высшее,0,гражданский брак,1,F,сотрудник,0,NaN,операции с жильем
4077,0,-7059.100220,45,высшее,0,гражданский брак,1,F,компаньон,1,194820.185757,сыграть свадьбу
10437,0,-2258.301455,50,среднее,1,женат / замужем,0,F,сотрудник,0,146768.250599,операции с коммерческой недвижимостью
9631,1,-1045.752744,23,среднее,1,женат / замужем,0,M,сотрудник,0,81734.568425,операции с коммерческой недвижимостью


Сразу видно множество проблем с данными:
- общий трудовой стаж может быть отрицательным или отсутствующим;
- тип образования имеет дубликаты по сути, но с разным написанием (капслок или нет);
- цель кредита может быть по сути одинаковой, но выражена синонимами. 

### Описание колонок

- `children` — количество детей в семье
- `days_employed` — общий трудовой стаж в днях
- `dob_years` — возраст клиента в годах
- `education` — уровень образования клиента
- `education_id` — идентификатор уровня образования
- `family_status` — семейное положение
- `family_status_id` — идентификатор семейного положения
- `gender` — пол клиента
- `income_type` — тип занятости
- `debt` — имел ли задолженность по возврату кредитов
- `total_income` — ежемесячный доход
- `purpose` — цель получения кредита

Колонки стоит переименовать: например, категориальные данные то имеют
суффикc `type` (`income_type`), то нет (`education`). Бинарным типа имеет смысл
дать приставку `is`/`has`


### Тип данных

In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Видно множество пропущенных значений в столбцах `days_employed`
и `total_income`, а также — местами — не совсем удачные автоматически типы.

- `children` — количество детей в семье
- `days_employed` — общий трудовой стаж в днях
- `dob_years` — возраст клиента в годах
- `education` — уровень образования клиента
- `education_id` — идентификатор уровня образования
- `family_status` — семейное положение
- `family_status_id` — идентификатор семейного положения
- `gender` — пол клиента
- `income_type` — тип занятости
- `debt` — имел ли задолженность по возврату кредитов
- `total_income` — ежемесячный доход
- `purpose` — цель получения кредита

Колонки стоит переименовать: например, категориальные данные то имеют
суффикc `type` (`income_type`), то нет (`education`). Бинарным типам имеет смысл
дать приставку `is_`/`has`. Часть имен просто не очень удачна (`dob_years`)

Покажем колонки с пустыми значениями:

In [54]:
columns_with_na = df.isna().sum()
columns_with_na = columns_with_na[columns_with_na > 0]
columns_with_na

days_employed    2174
total_income     2174
dtype: int64

## Первичная проверка и обработка данных

### Смена названий колонок

In [55]:
df = df.rename(columns={
    'dob_years': 'age',
    'education': 'education_type',
    'education_id': 'education_type_id',
    'family_status': 'marital_type',
    'family_status_id': 'marital_type_id',
    'debt': 'had_past_due_credits',
    'total_income': 'income',
    'purpose': 'purpose_type',
})

Выделим имена колонок в константы для удобства автокомплита (в т.ч. в массивах)

In [56]:
CHILDREN_COLUMN = 'children'
DAYS_EMPLOYED_COLUMN = 'days_employed'
AGE_COLUMN = 'age'
EDUCATION_TYPE_COLUMN = 'education_type'
EDUCATION_TYPE_ID_COLUMN = 'education_type_id'
MARITAL_TYPE_COLUMN = 'marital_type'
MARITAL_TYPE_ID_COLUMN = 'marital_type_id'
GENDER_COLUMN = 'gender'
INCOME_TYPE_COLUMN = 'income_type'
HAD_PAST_DUE_CREDITS_COLUMN = 'had_past_due_credits'
INCOME_COLUMN = 'income'
PURPOSE_TYPE_COLUMN = 'purpose_type'

### Доход (удаение пропусков)

In [57]:
df[INCOME_COLUMN].describe()

count    1.935100e+04
mean     1.674223e+05
std      1.029716e+05
min      2.066726e+04
25%      1.030532e+05
50%      1.450179e+05
75%      2.034351e+05
max      2.265604e+06
Name: income, dtype: float64

Отрицательных значений не видно

In [58]:
missing_share = df[INCOME_COLUMN].isna().sum() / len(df)
print(f'Доля пропущенных: {missing_share:.0%}')

Доля пропущенных: 10%


Заменим пропущенные (скорее всего, чтобы не указывать большой
или нелегальный доход) на медиану из-за большой разницы между значениями.
Пропущенных значений достаточно много, чтобы просто их удалить.

In [59]:
df[INCOME_COLUMN].fillna(df[INCOME_COLUMN].median(), inplace=True)

### Количество детей

Посмотрим на разные значения (как было видно выше, пропущенных тут нет)

In [60]:
df[CHILDREN_COLUMN].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

20 детей — похоже на ошибку (обратим внимание, но ничего с этим делать не будем).
Видно -1 количество детей — исправим их на положительные

In [61]:
df[CHILDREN_COLUMN] = df[CHILDREN_COLUMN].apply(lambda x: x if x >= 0 else -x)

### Стаж

In [62]:
df[DAYS_EMPLOYED_COLUMN].describe()

count     19351.000000
mean      63046.497661
std      140827.311974
min      -18388.949901
25%       -2747.423625
50%       -1203.369529
75%        -291.095954
max      401755.400475
Name: days_employed, dtype: float64

Видно, что есть отрицательные значения. Посмотрим на их долю

In [63]:
negative_share = len(df[df[DAYS_EMPLOYED_COLUMN] < 0]) / len(df)
print(f'Доля отрицательных: {negative_share:.0%}')

Доля отрицательных: 74%


Скорее всего, дело в том, что при подсчете перепутали местами дату начала
и конца — заменим отрицательные на положительные

In [64]:
df[DAYS_EMPLOYED_COLUMN] = df[DAYS_EMPLOYED_COLUMN].apply(
    lambda x: x if x >= 0 else -x)

Посмотрим на долю пропущенных значений

In [65]:
missing_share = df[DAYS_EMPLOYED_COLUMN].isna().sum() / len(df)
print(f'Доля пропущенных: {missing_share:.0%}')

Доля пропущенных: 10%


In [66]:
(
    df[df[DAYS_EMPLOYED_COLUMN].isna()]
    .groupby(INCOME_TYPE_COLUMN)[INCOME_TYPE_COLUMN]
    .count()
)

income_type
госслужащий         147
компаньон           508
пенсионер           413
предприниматель       1
сотрудник          1105
Name: income_type, dtype: int64

Данных достаточно много и пропущены они, скорее всего, по незнанию

In [67]:
df[DAYS_EMPLOYED_COLUMN].describe()

count     19351.000000
mean      66914.728907
std      139030.880527
min          24.141633
25%         927.009265
50%        2194.220567
75%        5537.882441
max      401755.400475
Name: days_employed, dtype: float64

Cтранно видеть очень большие значения. Посмотрим сколько таких чей стаж
неоправданно высок

In [68]:
invalid_employed_age = len(df[df[DAYS_EMPLOYED_COLUMN] / 365 > df[AGE_COLUMN] + 14]) / len(df)
print(f'Процент людей со слишком большим стажем {invalid_employed_age:.0%}')

Процент людей со слишком большим стажем 16%


Итого: 74% отрицательных и 10% пропущенных. Даже если исправить отрицательные,
16% со слишком большим стажем. Удалим эту колонку, верить данным нельзя.

In [69]:
df = df.drop(columns=DAYS_EMPLOYED_COLUMN)

### Возраст

In [70]:
df[AGE_COLUMN].describe()

count    21525.000000
mean        43.293380
std         12.574584
min          0.000000
25%         33.000000
50%         42.000000
75%         53.000000
max         75.000000
Name: age, dtype: float64

Проблем с возрастом не видно

### Образование

In [71]:
df[EDUCATION_TYPE_COLUMN].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education_type, dtype: int64

Видны неявные дубликаты из-за разных написаний — поправим

In [72]:
df[EDUCATION_TYPE_COLUMN] = df[EDUCATION_TYPE_COLUMN].str.lower()

### Семейное положение

In [73]:
df[MARITAL_TYPE_COLUMN].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: marital_type, dtype: int64

Проблем не видно

### Пол

Посмотрим на разные значения в столбце

In [74]:
df[GENDER_COLUMN].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Плохая запись только одна — удалим

In [75]:
df = df[df[GENDER_COLUMN] != 'XNA']

### Тип дохода

In [76]:
df[INCOME_TYPE_COLUMN].value_counts()

сотрудник          11119
компаньон           5084
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

Проблем не видно

### Наличие просрочек

In [77]:
df[HAD_PAST_DUE_CREDITS_COLUMN].value_counts()

0    19783
1     1741
Name: had_past_due_credits, dtype: int64

Проблем не видно

### Доход (отбросим дробную часть)

Округлим в меньшую сторону до целого

In [78]:
df[INCOME_COLUMN] = df[INCOME_COLUMN].astype('int')

### Цель кредита

In [79]:
df[PURPOSE_TYPE_COLUMN].value_counts().head()

свадьба                              797
на проведение свадьбы                777
сыграть свадьбу                      774
операции с недвижимостью             676
покупка коммерческой недвижимости    664
Name: purpose_type, dtype: int64

Видно большое количество неявных дубликатов, разберемся с ними позже

### Резюме по первичной обработке
1. Было 47 записей с -1 количеством детей (поправили на 1).
2. Отрицательные значения в стаже были заменены на положительные. Пропущенные заменены медианным значением.
3. В образовании были исправлены разные написания одних и тех же значений.
4. Один «пол» был неверным — запись удалена.
5. 10% пропущенных значений в доходе заменены медианным. Дробная часть отброшена.

## Нормализация данных

Выделим повторяющиеся данные (тип образования и семейное положение)

In [80]:
education_types = df[[EDUCATION_TYPE_COLUMN, EDUCATION_TYPE_ID_COLUMN]].copy()
education_types = education_types.drop_duplicates().reset_index(drop=True)

In [81]:
education_types

,education_type,education_type_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


In [82]:
marital_types = df[[MARITAL_TYPE_COLUMN, MARITAL_TYPE_ID_COLUMN]].copy()
marital_types = marital_types.drop_duplicates().reset_index(drop=True)

In [83]:
marital_types

,marital_type,marital_type_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


Удалим из таблицы строковые столбцы

In [84]:
df.drop(columns=[EDUCATION_TYPE_COLUMN, MARITAL_TYPE_COLUMN], inplace=True)

Вывод: выделили из таблицы повторяющиеся строковые значения в отдельные таблицы.

## Валидация данных

Проверим данные «на вшивость». Посмотрим на образование и возраст

### Образование

In [85]:
education_types

,education_type,education_type_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


In [86]:
def get_education_id(type: str) -> int:
    return (
        education_types
        .loc[education_types[EDUCATION_TYPE_COLUMN] == type,
             EDUCATION_TYPE_ID_COLUMN]
        .iloc[0])

In [87]:
def get_count(type_id: int, age_limit: int) -> int:
    return len(
        df[(df[EDUCATION_TYPE_ID_COLUMN] == type_id)
           & (df[AGE_COLUMN] <= age_limit)
           & (df[AGE_COLUMN] != 0)])

In [88]:
type_id = get_education_id('ученая степень')
print(f'Количество людей с невозможной учёной степенью: {get_count(type_id, 22)}')

type_id = get_education_id('высшее')
print(f'Количество людей с невозможным высшим образованием: {get_count(type_id, 19)}')

type_id = get_education_id('неоконченное высшее')
print(f'Количество людей с невозможным неоконченным высшим: {get_count(type_id, 17)}')

type_id = get_education_id('среднее')
print(f'Количество людей с невозможным средним образованием: {get_count(type_id, 16)}')



Количество людей с невозможной учёной степенью: 0
Количество людей с невозможным высшим образованием: 0
Количество людей с невозможным неоконченным высшим: 0
Количество людей с невозможным средним образованием: 0


Проблем с несоответствующему возрасту уровню образования не найдено

## Категоризация данных

### Доход

Введем 5 категорий дохода:
1. 0–30000 — `E`;
2. 30001–50000 — `D`;
3. 50001–200000 — `C`;
4. 200001–1000000 — `B`;
5. 1000001 и выше — `A`.

In [89]:
def get_income_category(income: int) -> str:
    if income <= 30_000:
        return 'E'
    if 30_000 < income <= 50_000:
        return 'D'
    if 50_000 < income <= 200_000:
        return 'C'
    if 200_000 < income <= 1_000_000:
        return 'B'
    return 'A'

In [90]:
TOTAL_INCOME_CATEGORY_COLUMN = 'total_income_category'
df[TOTAL_INCOME_CATEGORY_COLUMN] = df[INCOME_COLUMN].apply(get_income_category)

### Цель кредита

Введем следующие категории целей кредита:
- операции с автомобилем
- операции с недвижимостью
- проведение свадьбы
- получение образования

In [91]:
PURPOSE_CATEGORY_COLUMN = 'purpose_category'


def set_normalized_purpose(pattern: str, category: str) -> None:
    df.loc[df[PURPOSE_TYPE_COLUMN].str.contains(pattern),
           PURPOSE_CATEGORY_COLUMN] = category

In [92]:
set_normalized_purpose('авто', 'операции с автомобилем')
set_normalized_purpose('недвиж|жиль', 'операции с недвижимостью')
set_normalized_purpose('свадь', 'проведение свадьбы')
set_normalized_purpose('образ', 'получение образования')

 Проверим, что покрыли все цели кредита

In [93]:
print('Количество неразмеченных категорий', len(df[df[PURPOSE_CATEGORY_COLUMN].isna()]))

Количество неразмеченных категорий 0


## Количество детей и возврат кредита в срок

In [114]:
(df
 .groupby(CHILDREN_COLUMN)[HAD_PAST_DUE_CREDITS_COLUMN]
 .agg(['count', 'sum', 'mean'])
 .sort_values(by='mean', ascending=False))

,count,sum,mean
children,,,
20,76,8,0.105263
4,41,4,0.097561
2,2055,194,0.094404
1,4865,445,0.091470
3,330,27,0.081818
0,14148,1063,0.075134
5,9,0,0.000000


Если просто смотреть на детей без группировки, особой разницы нет.

Выделим две категории: есть дети или нет:

In [51]:
print(f'Без детей, не было просрочек {df[df[CHILDREN_COLUMN] == 0][HAD_PAST_DUE_CREDITS_COLUMN].mean():0.1%}')
print(f'Есть дети, не было просрочек {df[df[CHILDREN_COLUMN] > 0][HAD_PAST_DUE_CREDITS_COLUMN].mean():0.1%}')

Без детей, не было просрочек 7.5%
Есть дети, не было просрочек 9.2%


Вывод: бездетные на 1.7% «выгоднее»

## Семейное положение и возврат кредита в срок

In [112]:
(df[[HAD_PAST_DUE_CREDITS_COLUMN, MARITAL_TYPE_ID_COLUMN]]
 .merge(marital_types, on=MARITAL_TYPE_ID_COLUMN)
 .groupby(MARITAL_TYPE_COLUMN)
 .agg(['count', 'sum', 'mean'])['had_past_due_credits']
 .sort_values(by='mean', ascending=False))

,count,sum,mean
marital_type,,,
Не женат / не замужем,2813,274,0.097405
гражданский брак,4176,388,0.092912
женат / замужем,12380,931,0.075202
в разводе,1195,85,0.071130
вдовец / вдова,960,63,0.065625


Вывод: неженатые ненамного, но все же чаще имеют просрочки, чем люди в/бывшие в отношениях.

## Уровень дохода и возврат кредита в срок

In [113]:
(df
 .groupby(TOTAL_INCOME_CATEGORY_COLUMN)[HAD_PAST_DUE_CREDITS_COLUMN]
 .agg(['count', 'sum', 'mean'])
 .sort_values(by='mean', ascending=False))

,count,sum,mean
total_income_category,,,
E,22,2,0.090909
C,16087,1360,0.084540
A,25,2,0.080000
B,5040,356,0.070635
D,350,21,0.060000


Вывод: более-менее нормальная выборка только в двух категориях (B и С),
но и здесь видно, что люди с большим доходом реже имеют просрочки.

## Цели кредита и возврат в срок

In [115]:
(df
 .groupby(PURPOSE_CATEGORY_COLUMN)[HAD_PAST_DUE_CREDITS_COLUMN]
 .agg(['count', 'sum', 'mean'])
 .sort_values(by='mean', ascending=False))

,count,sum,mean
purpose_category,,,
операции с автомобилем,4315,403,0.093395
получение образования,4022,370,0.091994
проведение свадьбы,2348,186,0.079216
операции с недвижимостью,10839,782,0.072147


Вывод: недвижимость на 1-2 процентных пункта реже страдает от просрочек.

## Выводы

1. Во входных данных было много проблем:
- 47 записей с «-1» количеством детей (поправили на 1);
- стаж в данных был невалиден: 74% отрицательных чисел и 10% пропущенных. Даже если исправить отрицательные, 16% со слишком большим (невозможным) стажем. Эта колонку не стоит использовать в анализе без уточнения данных в источниках.
- в образовании были исправлены разные написания одних и тех же значений;
- один «пол» был неверным — запись удалена;
- 10% пропущенных значений в доходе заменены медианным. Дробная часть отброшена.

2. На возврат кредита в срок не очень сильно влияет (1-2 процентных пункта) ни количество детей, ни семейное положение, ни цель, ни уровень дохода. Для более
точного скоринга не стоит полагаться только на один такой параметр в отдельности.
Но если других вариантов нет, то анализ показывает очевидное:
- бездетные ненамного, но реже имеют просроки;
- незамужние ненамного, но чаще имеют просрочки по сравнению с людьми в текущих или бывших отношениях;
- люди с более высоким доходом ненамного, но реже имеют просрочки;
- люди, взявшие кредит на недвижимость, ненамного, но реже имеют просрочки.
- повторим: разница везде в 1-2 процентных пункта.